# Installations

In [2]:
 !pip install torch

In [ ]:
# Install required libraries
%pip install \
    transformers==4.27.2 \
    datasets==2.11.0  --quiet

# Importings

In [4]:
from datasets import load_dataset
from transformers import AutoModelForSeq2SeqLM
from transformers import AutoTokenizer
from transformers import GenerationConfig

# Data Sample

In [7]:
conv1 = '''

User: Hi there! I'm interested in exchanging my laptop for a newer model. It's in good condition, but I need something with better performance for my work. Do you have any options available?

Seller: Hello! Sure, we have a variety of laptops with different specifications. Can you provide more details about your current laptop, such as the model, specifications, and any additional features you're looking for in the new one?

User: Of course. It's a mid-range laptop with an Intel Core i5 processor, 8GB RAM, and a 256GB SSD. I'm looking for something with a more powerful processor and larger storage capacity.

Seller: Great! We have several options that meet your criteria. Let me show you a few models and discuss the possibility of a trade-in discount for your current laptop.'''
sum1 = """The user initiates a conversation expressing interest in exchanging their current laptop for a newer model with better performance. The seller engages in a dialogue, seeking details about the user's current laptop and their preferences for the new one. The user specifies the specifications they have and the desired upgrades. The seller then offers to show various laptop options and discusses the potential for a trade-in discount, aiming to facilitate a smooth product exchange."""

conv2="""User: Hi, I recently purchased a smartphone from your store, and I'm not satisfied with its performance. I've been experiencing frequent glitches and slow response times. Is there a possibility to return or exchange it for a different model?

Customer Service: Hello! I'm sorry to hear about the issues you're facing. We value your feedback. Can you provide more details about the specific problems you're encountering with the smartphone?

User: Certainly. The phone freezes often, and the applications take a long time to load. I've tried troubleshooting, but the issues persist, affecting my overall experience with the device.

Customer Service: I understand your concerns. Let me check our return and exchange policy for you. It typically involves a certain timeframe for returns and exchanges, and we may need to assess the device's condition. Have you kept the original packaging and accessories?"""

sum2="""
The user expresses dissatisfaction with a recently purchased smartphone and inquires about the possibility of a return or exchange due to performance issues. The customer service representative engages in a conversation, asking for specific details about the problems. The user describes freezing and slow response times. The representative acknowledges the concerns and mentions the return and exchange policy, indicating a need to assess the device's condition. The user's request for resolution is met with an exploration of the store's procedures for handling such cases."""

# Flan-T5 LLM - Model & Tokenizer

**Flan-T5** is an instruction-tuned version of **T5**, a popular encoder-decoder model. It is capable of performing various zero-shot NLP tasks such as text summarization, common sense reasoning, natural language inference, question answering, sentence and sentiment classification, translation, and pronoun resolution with appropriate prompting.

Flan-T5 is an instruction-tuned model and is capable of performing few-shot in-context learning tasks. The **Flan-T5** family of models is used in Amazon's Amazon Mechanical Turk platform.

- Encodeing and decoding for sample sentence

In [16]:
model_name='google/flan-t5-base'

model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)

In [8]:
sentence = "A large language model (LLM) is a type of artificial intelligence (AI) algorithm that uses deep learning techniques"
#sentence = "What time is it, Tom?"

sentence_encoded = tokenizer(sentence, return_tensors='pt')

sentence_decoded = tokenizer.decode(
        sentence_encoded["input_ids"][0],
        skip_special_tokens=True
    )

print('ENCODED SENTENCE:')
print(sentence_encoded["input_ids"][0])
print('\nDECODED SENTENCE:')
print(sentence_decoded)


ENCODED SENTENCE:
tensor([   71,   508,  1612,   825,    41, 10376,   329,    61,    19,     3,
            9,   686,    13,  7353,  6123,    41,  9822,    61, 12628,    24,
         2284,  1659,  1036,  2097,     1])

DECODED SENTENCE:
A large language model (LLM) is a type of artificial intelligence (AI) algorithm that uses deep learning techniques


# Prompt Engineering

In [9]:
def model_generation(model,prompt,config = None):
    tokenized = tokenizer(
        prompt,
        return_tensors = 'pt',
    ).input_ids
    if config is not None:
        generation=model.generate(
        tokenized,
        generation_config = generation_config)[0]
    else:
        generation = model.generate(
            tokenized,
            max_new_tokens = 50
        )[0]
    decoded = tokenizer.decode(
        generation,
        skip_special_tokens = True
    )
    return decoded

In [10]:
model_generation(model=model,prompt=conv1)

'Seller: Sure!'


# Instructional Prompts

In our earlier evaluation, we recognized the model's inherent capabilities; however, it struggled to grasp the core essence of the task at hand.

Now, we're equipped to bridge that understanding gap through a powerful technique called Instruction Fine-Tuning. Within this approach lies the art of Prompt Engineering and the finesse of Few-Shot Inferences. These tools will enable us to guide and support the model, ensuring it not only comprehends the task but excels in delivering precise and context-aware responses.

add Codeadd Markdown

# Zero Shot Inference

Without providing any task-specific examples, we entrust the model with the task, expecting it to autonomously comprehend the problem and generate the desired outcomes. It's a test of the model's inherent understanding and adaptability.

In [11]:
prompt = f"""
Dialog:

{conv1}

Whats going on?"""

In [12]:
model_completion = model_generation(model=model,prompt=prompt)

In [13]:
model_completion

"User: Hi there! I'm interested in trading in my laptop for a newer model."

# One Shot Learning

In [14]:
#my code
prompt=f"""
Dialog:
{conv1}

what's going on ?
{sum1}

Dialog:
{conv2}

what's going on ?
"""
model_completion = model_generation(model = model,prompt=prompt)

In [15]:
model_completion

"User: I'm not satisfied with the performance of the smartphone. I've tried troubleshooting, but the issues persist. Customer Service will return or exchange the device."

From above results we can say that one shot is giving comparitively better results than zero shot.